# Search AiiDA Database for Slab Models

In [ ]:
%aiida

In [ ]:
# General imports
#import ase.io
#from base64 import b64encode
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import datetime
#from tempfile import NamedTemporaryFile
    
# Custom imports
from apps.surfaces.widgets import  search_completed

In [ ]:
############################   START OF PREPROCESSING   ###############################

In [ ]:
PREPROCESS_VERSION = 0.96



In [ ]:
############################   END OF PREPROCESSING   ###############################

In [ ]:
def search():
    
    results.value = "preprocessing..."
    search_completed.preprocess_newbies(preprocess_v=PREPROCESS_VERSION,wlabel='SlabGeoOptWorkChain',clabel='slab_geo_opt')
    try:
        import apps.scanning_probe.common
        apps.scanning_probe.common.preprocess_spm_calcs(workchain_list = ['STMWorkChain', 'PdosWorkChain', 'AfmWorkChain'])
    except:
        print("Warning: scanning_probe app not found, skipping spm preprocess.")
    
    results.value = "searching..."
    
    # html table header
    html  = '<style>#aiida_results td,th {padding: 2px}</style>' 
    html += '<table border=1 id="aiida_results" style="margin:10px;"><tr>'
    html += '<th>PK</th>'
    html += '<th>Creation Time</th>'
    html += '<th>Formula</th>'
    html += '<th>CalcName</th>'
    html += '<th>Energy</th>'
    html += '<th>Structure</th>'
    html += '<th>Extra calculations</th>'
    html += '</tr>'

    # query AiiDA database
    filters = {}
    filters['label'] = 'SlabGeoOptWorkChain'
    filters['extras.preprocess_version'] = PREPROCESS_VERSION
    filters['extras.preprocess_successful'] = True
    filters['extras.obsolete'] = False
    
    pk_list = inp_pks.value.strip().split()
    if pk_list:
        filters['id'] = {'in': pk_list}
        
    formula_list = inp_formula.value.strip().split()
    if inp_formula.value:
        filters['extras.formula'] = {'in': formula_list}
        
    if len(text_description.value) > 1:
        filters['description'] = {'like': '%{}%'.format(text_description.value)}
    
    try: # If the date range is valid, use it for the search
        start_date = datetime.datetime.strptime(date_start.value, '%Y-%m-%d')
        end_date = datetime.datetime.strptime(date_end.value, '%Y-%m-%d') + datetime.timedelta(hours=24)
    except ValueError: # Otherwise revert to the standard (i.e. last 10 days)
        end_date = datetime.datetime.now()
        start_date = end_date - datetime.timedelta(days=20)

        date_start.value = start_date.strftime('%Y-%m-%d')
        date_end.value = end_date.strftime('%Y-%m-%d')
    
    filters['ctime'] = {'and':[{'<=': end_date},{'>': start_date}]}
    
    qb = QueryBuilder()        
    qb.append(WorkChainNode, filters=filters)
    qb.order_by({WorkChainNode:{'ctime':'desc'}})

    for i, node_tuple in enumerate(qb.iterall()):
        node = node_tuple[0]
        thumbnail = node.extras['thumbnail']
        description = node.extras['structure_description']
        opt_structure_uuid = node.extras['opt_structure_uuid']
        
        ## Find all extra calculations done on the optimized geometry
        extra_calc_links = ""
        opt_structure = load_node(opt_structure_uuid)
        st_extras = opt_structure.extras
        
        ### --------------------------------------------------
        ### add links to SPM calcs
        try:
            import apps.scanning_probe.common
            extra_calc_links += apps.scanning_probe.common.create_viewer_link_html(st_extras, "../../")
        except Exception as e:
            pass
        ### --------------------------------------------------
        
        extra_calc_area = "<div id='wrapper' style='overflow-y:auto; height:100px; line-height:1.5;'> %s </div>" % extra_calc_links
        
        # append table row
        html += '<tr>'
        html += '<td>%d</td>' % node.pk
        html += '<td>%s</td>' % node.ctime.strftime("%Y-%m-%d %H:%M")
        html += '<td>%s</td>' % node.extras['formula']
        html += '<td>%s</td>' % node.description
        html += '<td>%f</td>' % node.extras['energy']
        html += '<td><a target="_blank" href="../export_structure.ipynb?uuid=%s">'%opt_structure_uuid
        html += '<img width="100px" src="data:image/png;base64,%s" title="PK%d: %s"></a></td>' % (thumbnail, opt_structure.pk, description)
        html += '<td>%s</td>' % extra_calc_area
        html += '</td>'
        html += '</tr>'

    html += '</table>'
    html += 'Found %d matching entries.<br>'%qb.count()

    results.value = html

In [ ]:
# search UI
style = {"description_width":"150px"}
layout = ipw.Layout(width="600px")
inp_pks = ipw.Text(description='PKs', placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)
inp_formula = ipw.Text(description='Formulas:', placeholder='e.g. C44H16 C36H4', layout=layout, style=style)
text_description = ipw.Text(description='Calculation Name: ',
                            placeholder='e.g. keywords',
                            layout=layout, style=style)

# ---------
# date selection
dt_now = datetime.datetime.now()
dt_from = dt_now - datetime.timedelta(days=20)
date_start = ipw.Text(value=dt_from.strftime('%Y-%m-%d'),
                           description='From: ',
                           style={'description_width': '60px'}, layout={'width': '225px'})

date_end = ipw.Text(value=dt_now.strftime('%Y-%m-%d'),
                          description='To: ',
                          style={'description_width': '60px'}, layout={'width': '225px'})

date_text = ipw.HTML(value='<p>Select the date range:</p>', width="150px")
# ---------

search_crit = [inp_pks, inp_formula, text_description, ipw.HBox([date_text, date_start, date_end])]

In [ ]:
def on_click(b):
    with info_out:
        clear_output()
        search()

button = ipw.Button(description="Search")
button.on_click(on_click)
results = ipw.HTML()
info_out = ipw.Output()
app = ipw.VBox(children=search_crit + [button, results, info_out])
display(app)